# Petit Photoshop

Importer les librairies suivantes :

In [3]:
import tkinter as tk
import PIL as pil
from PIL import Image
from PIL import ImageTk 
from tkinter import filedialog
from tkinter import simpledialog

Ajouter une fonction `nbrCol(matrice)` qui retourne le nombre de colonnes d'une matrice 
et une fonction `nbrLig(matrice)` qui retourne le nombre de lignes d'une matrice

In [4]:
def nbrCol(matrice):
    return(len(matrice[0]))

def nbrLig(matrice):
    return len(matrice)

Ajouter la fonction suivante à votre code, on ne demande pas de la comprendre en détail.
Elle permet de stocker la matrice de pixels passée en paramètre `matPix` dans un fichier spécifié par `filename`.

In [5]:
def saving(matPix, filename):
    toSave=pil.Image.new("RGBA",(nbrCol(matPix),nbrLig(matPix)))
    for i in range(nbrCol(matPix)):
        for j in range(nbrLig(matPix)):
            toSave.putpixel((i,j),matPix[j][i])
    toSave.save(filename)

Ajouter la fonction `loading` à votre code. Là non plus on ne demande pas de comprendre la fonction en détail.
Elle prend une image en entrée et retourne une matrice de pixels.

In [6]:
def loading(filename):
    toLoad=pil.Image.open(filename)
    mat=[[(255,255,255,255)]*toLoad.size[0] for k in range(toLoad.size[1])]
    for i in range(toLoad.size[1]):
        for j in range(toLoad.size[0]):
            mat[i][j]=toLoad.getpixel((j,i))
    return mat

Ajouter la fonction `charger` qui permet d'afficher une image dans un canevas dans la colonne 1 d'une grid.

In [7]:
create=True
nomImgCourante=""
nomImgDebut = ""

def charger(widg):
    global create
    global photo
    global img
    global canvas
    global dessin
    global nomImgCourante
    global nomImgDebut
    filename= filedialog.askopenfile(mode='rb', title='Choose a file')
    img = pil.Image.open(filename)
    nomImgCourante=filename.name
    nomImgDebut = filename.name
    photo = ImageTk.PhotoImage(img)
    if create:    
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin = canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=4,columnspan=2)
        create=False
        
    else:
        canvas.grid_forget()
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin=canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=18,columnspan=2)


Ajouter la fonction `modify(matrice)` qui permet de modifier une matrice de pixels au sein même d'un canevas créé par la methode `charger()`, et la fonction callback `reaffiche` qui permet de réafficher l'image de départ après lui avoir appliqué des effets.

In [8]:
def modify(matrice):
    global imgModif
    global nomImgCourante
    saving(matrice,"modif.png")
    imgModif=ImageTk.PhotoImage(file="modif.png")
    canvas.itemconfigure(dessin, image=imgModif)
    nomImgCourante="modif.png"

def reaffiche():
    global imgDebut
    global nomImgCourante
    if not create :
        imgDebut=ImageTk.PhotoImage(file=nomImgDebut)
        canvas.itemconfigure(dessin, image=imgDebut)
        nomImgCourante = nomImgDebut
    

Créez une fenêtre tkinter appelée "Mon Petit Photoshop", dans laquelle vous mettrez un bouton pour chacune des fonctions de traitement d'image que vous allez écrire. Vous mettrez égalment un bouton pour *charger* une image, un bouton pour *quitter la fenêtre* et un bouton pour *revenir à l'image de départ* en utilisant la fonction callback `reaffiche`. Vous ajouterez également un label contenant votre nom, prénom et numéro d'étudiant. Vous pourrez ajouter un bouton pour sauvegarder l'image, éventuellement sous un nouveau nom. ATTENTION : L'image est placée grâce à la méthode `grid` et elle couvre les colonnes 1 et 2 et les lignes 0,1,2 et 3.

Mise en Page 
1. Vous placerez les boutons pour les fonctions de traitement d'image dans la colonne 0 d'une grid respectivement
    aux lignes 0,1,2
1. Le bouton `Retour` (pour revenir à l'image de départ) sera placé dans la colonne 0 à l'avant dernière ligne
1. Le bouton `Charger` sera placé dans la colonne 0 dans la dernière ligne
1. Le bouton `Quitter` sera placé dans la dernière colonne et la dernière ligne
1. Le label sera placé juste sous l'image, entre les boutons `Charger` et `Quitter`
 
Si tout fonctionne, regardez la doc tkinter pour rendre ça un peu plus "joli".

In [9]:
def filtre_vert():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(0,mat[i][j][1],0,255)
    modify(mat)
            
def negatif():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(255-mat[i][j][0],255-mat[i][j][1],255-mat[i][j][2],255)
    modify(mat)
            
def symetrique():
    mat=loading(nomImgCourante)
    matSym=[[(0,0,0,0)]*nbrCol(mat) for k in range(nbrLig(mat))]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            matSym[i][nbrCol(mat)-1-j]=mat[i][j]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=matSym[i][j]
    modify(mat)

In [10]:
def luminosite(r,g,b):
    l =  int(0.2125*r + 0.0721*b + 0.7154* g)
    return(l)


def gris():
    #On utilisera la conversion CIE709 qui permet de calculer la teinte de gris qui va être affichée dans le pixel
    #La teinte affichée est : gris=0,2125*rouge + 0,0721*bleu + 0,7154*vert
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            l = luminosite(mat[i][j][0],mat[i][j][1],mat[i][j][2])
            mat[i][j] = (l,l,l,255)
            # calcul de la teinte de gris du pixel (CIE709)
    modify(mat)


def noirBlanc():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            l = luminosite(mat[i][j][0],mat[i][j][1],mat[i][j][2])
            mat[i][j] = (0,0,0) if l < 127 else (255,255,255)
            # un pixel est blanc quand sa luminosité est > à 127, noir sinon
    modify(mat)

In [11]:
def shrink():
    mat=loading(nomImgCourante)
    #créer une matrice de largeur et hauteur deux fois plus petite 
    out =[]
    for i in range(nbrLig(mat)):
        if i % 2 == 1:
            continue
        l = []
        for j in range(nbrCol(mat)):
            try :
                if j % 2 == 0:
                    r = int((mat[i][j][0]+mat[i][j+1][0]+ mat[i+1][j][0] + mat[i+1][j+1][0])// 4)
                    g = int((mat[i][j][1]+mat[i][j+1][1]+ mat[i+1][j][1] + mat[i+1][j+1][1])// 4)
                    b = int((mat[i][j][2]+mat[i][j+1][2]+ mat[i+1][j][2] + mat[i+1][j+1][2])// 4)
                    l.append((r,g,b,255))
            except: 
                pass
        out.append(l)
    mat = list(out)
    modify(mat)
# def shrink():
#     mat=loading(nomImgCourante)
#     #créer une matrice de largeur et hauteur deux fois plus petite 
#     matshrink=[[(0,0,0,255)] *(nbrCol(mat)//2) for j in range(nbrLig(mat)//2)] 
#     for i in range(nbrLig(matshrink)):
#         for j in range(nbrCol(matshrink)):
#             pixRouge=(mat[2*i][2*j][0]+mat[2*i][2*j+1][0]+mat[2*i+1][2*j][0]+mat[2*i+1][2*j+1][0])//4
#             pixVert=(mat[2*i][2*j][1]+mat[2*i][2*j+1][1]+mat[2*i+1][2*j][1]+mat[2*i+1][2*j+1][1])//4
#             pixBleu=(mat[2*i][2*j][2]+mat[2*i][2*j+1][2]+mat[2*i+1][2*j][2]+mat[2*i+1][2*j+1][2])//4
#             matshrink[i][j]=(pixRouge,pixVert,pixBleu,255)
#     modify(matshrink)

In [12]:
def rotate():
    l = []
    mat=loading(nomImgCourante)
    for i in range (nbrCol(mat)):
        a = []
        for j in range (nbrLig(mat)):
            a. append(mat[j][i])
        a.reverse()
        l.append(a)
    mat = l

    modify(mat)

In [13]:
import random as rd
create=True

def qitter_fenetre():
    fenetre.destroy()


def zoom():
    mat=loading(nomImgCourante)
    #créer une matrice de largeur et hauteur deux fois plus grande 
    mat_zoom = []
    for i in range(nbrLig(mat)):
        a = []
        for j in range(nbrCol(mat)):
            for k in range (2):
                a.append(mat[i][j])
        for l in range (2):
            mat_zoom.append(a)
    mat = list(mat_zoom)
    modify(mat)


def dithering():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            l = luminosite(mat[i][j][0],mat[i][j][1],mat[i][j][2])
            n = rd.randint(0,255)
            mat[i][j] = (0,0,0) if l < n else (255,255,255)
            # un pixel est blanc quand sa luminosité est > à 127, noir sinon
    modify(mat)


#Fonctions auxiliaires 

    
fenetre = tk.Tk()
fenetre.title("Mon petit photoshop")
#Création des Widgets
bouton_retour = tk.Button(fenetre, text= "retour", command= reaffiche)
bouton_charger = tk.Button(fenetre, text= "charger", command= lambda : charger(fenetre) )
bouton_quitter = tk.Button(fenetre, text= "quitter", command= qitter_fenetre )
bouton_filtre_vert = tk.Button(fenetre,text="filtre vert", command=filtre_vert )
bouton_negatif = tk.Button(fenetre,text="négatif", command= negatif )
bouton_symetrie = tk.Button(fenetre,text="symétrie", command= symetrique )
bouton_gris = tk.Button(fenetre, text="gris", command= gris)
bouton_noir = tk.Button(fenetre, text="noir", command= dithering)
bouton_zoom = tk.Button(fenetre, text="zoom", command= zoom)
bouton_shrink = tk.Button(fenetre, text="shrink", command= shrink)
bouton_rotate = tk.Button(fenetre, text="rotate", command= rotate )


#Positionnement des Widgets

bouton_retour.grid(column=0 , row =1)
bouton_charger.grid(column= 0, row= 0)
bouton_quitter.grid(column=3)
bouton_filtre_vert.grid(column=0, row= 4)
bouton_negatif.grid(column= 0, row=2)
bouton_symetrie.grid(column=0, row= 3)
bouton_gris.grid(column=3)
bouton_noir.grid(column=3,row=0)
bouton_zoom.grid(column=3,row=1)
bouton_shrink.grid(column=3,row=2)
bouton_rotate.grid(column=3,row=3)





#Lancement de la boucle 
fenetre.mainloop()

Modifiez les fonctions du TD précédent pour qu'elles deviennent des fonctions callback qui s'appliquent directement sur l'image dans le canevas.

Faire une fonction qui permet de transformer une image en couleur en une image en niveau de gris.

Faire une fonction qui permet de transformer une image en couleur en une image en noir et blanc.

Donner une fonction qui multiplie par 4 la définition d'une image (chaque pixel donne un carré de 4 pixels).

Donner une fonction qui divise par 4 la définition d'une image. Chaque carré de 4 pixels est remplacé par 1 pixel dont la valeur est la moyenne de ces 4 pixels.

Combiner les fonctions `zoom` et `shrink` pour obtenir un effet poster (effet de perte de résolution).

In [ ]:
def poster():
    mat=loading(nomImgCourante)
    modify(mat)

Donner une fonction qui effectue une rotation d'une image vers la droite.

Voici quelques fonctions que vous pouvez implémenter si vous en avez le temps:
1. modification de la luminosité d'une image (avec un slider pour la sélection)
1. flou de l'image, chaque pixel est remplacé par la moyenne de ses voisins et de lui-même
1. bruitage de l'image, chaque pixel est modifié par une valeur aléatoire
1. implémenter une méthode de dithering pour obtenir une image en noir et blanc plus subtile (au lieu de prendre un seuil
à 127, on tire aléaoirement la valeur du pixel avec une probabilité qui dépend de sa luminosité)
1. mode stylo, qui permet avec la souris d'écrire des pixels noirs sur l'image 